In [30]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


## Conexión a la base de datos

In [31]:
import mysql.connector as mc
from mysql.connector import Error

HOST = "localhost"
PORT = 3306
USER = "root"
PASSWORD = "Aquí ponen su contraseña de mysql, sino no jala xd"
DB_DW = "dw_multas"
DB_SRC_MULTAS = "multas_transito"
DB_REG_VEH = "registro_vehiculos"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

def scalar(cur, sql):
    cur.execute(sql)
    return cur.fetchone()[0]


## Cargar DimOficial

In [32]:
import mysql.connector as mc
from mysql.connector import Error

DB_DW="dw_multas"; DB_SRC="multas_transito"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

try:
    dw = connect(DB_DW); cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)

    # Conteo antes
    cur.execute("SELECT COUNT(*) FROM DimOficial;")
    before = cur.fetchone()[0]
    print("📊 DimOficial (ANTES):", before)

    # Inserción idempotente
    sql = f"""
    INSERT INTO DimOficial (id_oficial, nombre, rango)
    SELECT o.id_oficial, o.nombre, o.rango
    FROM {DB_SRC}.oficial o
    LEFT JOIN DimOficial d ON d.id_oficial = o.id_oficial
    WHERE d.id_oficial IS NULL;
    """
    cur.execute(sql); dw.commit()

    # Conteo después + preview
    cur.execute("SELECT COUNT(*) FROM DimOficial;")
    after = cur.fetchone()[0]
    print("📊 DimOficial (DESPUÉS):", after, " | Insertadas:", after - before)

    cur.execute("SELECT id_oficial, nombre, rango FROM DimOficial ORDER BY id_oficial LIMIT 5;")
    for r in cur.fetchall():
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
    except:
        pass


✅ Conectado a dw_multas; server: 9.4.0
📊 DimOficial (ANTES): 120
📊 DimOficial (DESPUÉS): 120  | Insertadas: 0
(1, 'Carlos Cruz', 'Oficial')
(2, 'Mateo Santos', 'Sargento')
(3, 'Diego Reyes', 'Oficial')
(4, 'Oliver Sanchez', 'Oficial')
(5, 'Santiago Sanchez', 'Sargento')


## Cargar DimTipoMulta (categoría: conducción)

In [33]:
try:
    dw = connect(DB_DW)
    cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)

    cur.execute("SELECT COUNT(*) FROM DimTipoMulta WHERE categoria='conduccion';")
    before = cur.fetchone()[0]
    print("📊 DimTipoMulta (conducción) ANTES:", before)


    sql_insert = f"""
    INSERT INTO {DB_DW}.DimTipoMulta (categoria, id_tipo_natural, violacion, tarifa)
    SELECT 'conduccion', t.id_tipo_multa_conduccion, t.violacion, t.tarifa
    FROM {DB_SRC_MULTAS}.tipomultaconduccion t
    LEFT JOIN {DB_DW}.DimTipoMulta d
      ON d.categoria = 'conduccion'
     AND d.id_tipo_natural = t.id_tipo_multa_conduccion
    WHERE d.tipo_multa_key IS NULL;
    """
    cur.execute(sql_insert)
    dw.commit()


    cur.execute("SELECT COUNT(*) FROM DimTipoMulta WHERE categoria='conduccion';")
    after = cur.fetchone()[0]
    print("📊 DimTipoMulta (conducción) DESPUÉS:", after)
    print("🟣 Filas insertadas en esta corrida:", after - before)


    cur.execute("""
        SELECT tipo_multa_key, categoria, id_tipo_natural, violacion, tarifa
        FROM DimTipoMulta
        WHERE categoria='conduccion'
        ORDER BY id_tipo_natural
        LIMIT 10;
    """)
    rows = cur.fetchall()
    print("\n🔎 Preview (TOP 10 - conducción):")
    for r in rows:
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
        print("\n🔒 Conexión cerrada.")
    except:
        pass

✅ Conectado a dw_multas; server: 9.4.0
📊 DimTipoMulta (conducción) ANTES: 50
📊 DimTipoMulta (conducción) DESPUÉS: 50
🟣 Filas insertadas en esta corrida: 0

🔎 Preview (TOP 10 - conducción):
(1, 'conduccion', 'D001', 'Exceso de velocidad', Decimal('2455.00'))
(2, 'conduccion', 'D002', 'Conducir bajo influencia (DUI)', Decimal('1000.00'))
(3, 'conduccion', 'D003', 'No respetar señal de alto', Decimal('2995.00'))
(4, 'conduccion', 'D004', 'Pasarse el semáforo en rojo', Decimal('3960.00'))
(5, 'conduccion', 'D005', 'Conducción temeraria', Decimal('4090.00'))
(6, 'conduccion', 'D006', 'Uso de celular al conducir', Decimal('2815.00'))
(7, 'conduccion', 'D007', 'No portar cinturón de seguridad', Decimal('675.00'))
(8, 'conduccion', 'D008', 'Circular en sentido contrario', Decimal('620.00'))
(9, 'conduccion', 'D009', 'Rebasar en zona prohibida', Decimal('4600.00'))
(10, 'conduccion', 'D010', 'No respetar límite de carril confinado', Decimal('4065.00'))

🔒 Conexión cerrada.


## DimTipoMulta (categoría: estacionamiento)

In [34]:

try:
    dw = connect(DB_DW)
    cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)


    cur.execute("SELECT COUNT(*) FROM DimTipoMulta WHERE categoria='estacionamiento';")
    before = cur.fetchone()[0]
    print("📊 DimTipoMulta (estacionamiento) ANTES:", before)


    sql_insert = f"""
    INSERT INTO {DB_DW}.DimTipoMulta (categoria, id_tipo_natural, violacion, tarifa)
    SELECT 'estacionamiento',
           CAST(t.id_tipo_multa_estacionamiento AS CHAR),
           t.violacion,
           t.tarifa
    FROM {DB_SRC_MULTAS}.tipomultaestacionamiento t
    LEFT JOIN {DB_DW}.DimTipoMulta d
      ON d.categoria = 'estacionamiento'
     AND d.id_tipo_natural = CAST(t.id_tipo_multa_estacionamiento AS CHAR)
    WHERE d.tipo_multa_key IS NULL;
    """
    cur.execute(sql_insert)
    dw.commit()


    cur.execute("SELECT COUNT(*) FROM DimTipoMulta WHERE categoria='estacionamiento';")
    after = cur.fetchone()[0]
    print("📊 DimTipoMulta (estacionamiento) DESPUÉS:", after)
    print("🟣 Filas insertadas en esta corrida:", after - before)

    # Preview top 10
    cur.execute("""
        SELECT tipo_multa_key, categoria, id_tipo_natural, violacion, tarifa
        FROM DimTipoMulta
        WHERE categoria='estacionamiento'
        ORDER BY id_tipo_natural
        LIMIT 10;
    """)
    rows = cur.fetchall()
    print("\n🔎 Preview (TOP 10 - estacionamiento):")
    for r in rows:
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
        print("\n🔒 Conexión cerrada.")
    except:
        pass

✅ Conectado a dw_multas; server: 9.4.0
📊 DimTipoMulta (estacionamiento) ANTES: 43
📊 DimTipoMulta (estacionamiento) DESPUÉS: 43
🟣 Filas insertadas en esta corrida: 0

🔎 Preview (TOP 10 - estacionamiento):
(64, 'estacionamiento', 'P001', 'Estacionarse en lugar para discapacitados', Decimal('1450.00'))
(65, 'estacionamiento', 'P002', 'Estacionarse en doble fila', Decimal('1460.00'))
(66, 'estacionamiento', 'P003', 'Estacionarse sobre banqueta', Decimal('985.00'))
(67, 'estacionamiento', 'P004', 'Estacionarse en zona prohibida', Decimal('380.00'))
(68, 'estacionamiento', 'P005', 'Rebasar tiempo de parquímetro', Decimal('625.00'))
(69, 'estacionamiento', 'P006', 'No pagar parquímetro', Decimal('315.00'))
(70, 'estacionamiento', 'P007', 'Estorbar cochera', Decimal('835.00'))
(71, 'estacionamiento', 'P008', 'Estacionarse frente a hidrante', Decimal('2110.00'))
(72, 'estacionamiento', 'P009', 'Estacionarse en esquina', Decimal('2655.00'))
(73, 'estacionamiento', 'P010', 'Estacionarse en paso p

## Cargar DimVehiculo

In [35]:
DB_DW="dw_multas"
DB_REG="registro_vehiculos"  
TBL_REG="vehicle_registration"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

try:
    dw = connect(DB_DW)
    cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)


    cur.execute("SELECT COUNT(*) FROM DimVehiculo;")
    before = cur.fetchone()[0]
    print("📊 DimVehiculo (ANTES):", before)


    sql_insert = f"""
    INSERT INTO {DB_DW}.DimVehiculo (placas, marca, modelo, vehicle_year)
    SELECT g.placas, g.marca, g.modelo, g.vehicle_year
    FROM (
      SELECT UPPER(TRIM(v.vehicleLPN)) AS placas,
             MAX(v.vehicle_make)                         AS marca,
             MAX(v.vehicle_model)                        AS modelo,
             MAX(CAST(v.vehicle_year AS SIGNED))         AS vehicle_year
      FROM {DB_REG}.{TBL_REG} v
      WHERE v.vehicleLPN IS NOT NULL
        AND v.vehicleLPN <> ''
      GROUP BY UPPER(TRIM(v.vehicleLPN))
    ) AS g
    LEFT JOIN {DB_DW}.DimVehiculo d
      ON d.placas = g.placas
    WHERE d.vehiculo_key IS NULL;
    """
    cur.execute(sql_insert)
    dw.commit()
    # 3) Conteo después
    cur.execute("SELECT COUNT(*) FROM DimVehiculo;")
    after = cur.fetchone()[0]
    print("📊 DimVehiculo (DESPUÉS):", after)
    print("🟣 Filas insertadas en esta corrida:", after - before)


    cur.execute("""
        SELECT vehiculo_key, placas, marca, modelo, vehicle_year
        FROM DimVehiculo
        ORDER BY vehiculo_key
        LIMIT 10;
    """)
    print("\n🔎 Preview TOP 10 (DimVehiculo):")
    for r in cur.fetchall():
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
        print("\n🔒 Conexión cerrada.")
    except:
        pass

✅ Conectado a dw_multas; server: 9.4.0
📊 DimVehiculo (ANTES): 732
📊 DimVehiculo (DESPUÉS): 732
🟣 Filas insertadas en esta corrida: 0

🔎 Preview TOP 10 (DimVehiculo):
(64, 'YW43191', 'Mazda', 'CX-30', 2014)
(65, 'CA2493', 'Ford', 'Focus', 2010)
(66, 'QE34124', 'Mazda', 'CX-30', 2013)
(67, 'NE79762', 'Nissan', 'Rogue', 2018)
(68, 'YL20293', 'Honda', 'Civic', 2022)
(69, 'CI06022', 'Hyundai', 'Elantra', 2024)
(70, 'MP51106', 'Volkswagen', 'Polo', 2019)
(71, 'NY2750', 'Mazda', 'CX-5', 2002)
(72, 'GDL8422', 'Honda', 'Accord', 2006)
(73, 'VP51149', 'Peugeot', '3008', 2017)

🔒 Conexión cerrada.


In [36]:
import mysql.connector as mc
from mysql.connector import Error

DB_DW="dw_multas"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

try:
    dw = connect(DB_DW); cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)


    try:
        cur.execute("""
            ALTER TABLE DimVehiculo
            ADD COLUMN placas_norm VARCHAR(32)
            GENERATED ALWAYS AS (REGEXP_REPLACE(UPPER(placas), '[^A-Z0-9]', '')) STORED;
        """)
        print("🛠️ Columna placas_norm creada.")
    except Exception as e:
        print("ℹ️ Aviso:", e)

    try:
        cur.execute("CREATE UNIQUE INDEX uq_dimvehiculo_placas_norm ON DimVehiculo (placas_norm);")
        print("🛠️ Índice único sobre placas_norm creado.")
    except Exception as e:
        print("ℹ️ Aviso índice:", e)

    dw.commit()

    cur.execute("SELECT vehiculo_key, placas, placas_norm FROM DimVehiculo LIMIT 5;")
    for r in cur.fetchall():
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
    except:
        pass


✅ Conectado a dw_multas; server: 9.4.0
ℹ️ Aviso: 1060 (42S21): Duplicate column name 'placas_norm'
ℹ️ Aviso índice: 1061 (42000): Duplicate key name 'uq_dimvehiculo_placas_norm'
(64, 'YW43191', 'YW43191')
(65, 'CA2493', 'CA2493')
(66, 'QE34124', 'QE34124')
(67, 'NE79762', 'NE79762')
(68, 'YL20293', 'YL20293')


In [37]:
import mysql.connector as mc
from mysql.connector import Error

DB_DW="dw_multas"; DB_SRC="multas_transito"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

try:
    dw = connect(DB_DW); cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)


    cur.execute("SELECT COUNT(*) FROM FactMulta;")
    before = cur.fetchone()[0]
    print("📊 FactMulta (ANTES):", before)

    sql = f"""
    INSERT INTO FactMulta
      (id_multa, time_key, oficial_key, conductor_key, vehiculo_key, tipo_multa_key, importe, multa_count)
    SELECT
      mc.id_multa,
      CAST(DATE_FORMAT(mc.fecha,'%Y%m%d') AS UNSIGNED) AS time_key,
      dof.oficial_key,
      dco.conductor_key,
      dv.vehiculo_key,
      dtm.tipo_multa_key,
      dtm.tarifa,
      1
    FROM {DB_SRC}.multaconduccion mc
    JOIN DimOficial dof
      ON dof.id_oficial = mc.id_oficial
    LEFT JOIN DimConductor dco
      ON dco.licencia_id = mc.licencia_id
    JOIN DimVehiculo dv
      ON dv.placas_norm = REGEXP_REPLACE(UPPER(mc.placas), '[^A-Z0-9]', '')
    JOIN DimTipoMulta dtm
      ON dtm.categoria='conduccion'
     AND dtm.id_tipo_natural = CAST(mc.id_tipo_multa_conduccion AS CHAR)
    JOIN DimTiempo dti
      ON dti.time_key = CAST(DATE_FORMAT(mc.fecha,'%Y%m%d') AS UNSIGNED)
    LEFT JOIN FactMulta fm
      ON fm.id_multa = mc.id_multa
    WHERE fm.fact_multa_id IS NULL;
    """
    cur.execute(sql); dw.commit()


    cur.execute("SELECT COUNT(*) FROM FactMulta;")
    after = cur.fetchone()[0]
    print("📊 FactMulta (DESPUÉS):", after)
    print("🟣 Filas insertadas (conducción):", after - before)


    cur.execute("""
        SELECT id_multa, time_key, oficial_key, conductor_key, vehiculo_key, tipo_multa_key, importe
        FROM FactMulta
        ORDER BY fact_multa_id DESC
        LIMIT 5;
    """)
    print("\n🔎 Preview últimas 5:")
    for r in cur.fetchall():
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
        print("\n🔒 Conexión cerrada.")
    except:
        pass


✅ Conectado a dw_multas; server: 9.4.0
📊 FactMulta (ANTES): 815
📊 FactMulta (DESPUÉS): 815
🟣 Filas insertadas (conducción): 0

🔎 Preview últimas 5:
('900', 20220527, 52, None, 388, 67, Decimal('380.00'))
('896', 20210704, 77, None, 730, 101, Decimal('980.00'))
('894', 20240908, 19, None, 636, 74, Decimal('1385.00'))
('868', 20210306, 66, None, 313, 98, Decimal('1890.00'))
('847', 20221128, 23, None, 79, 103, Decimal('2070.00'))

🔒 Conexión cerrada.


## Cargar DimConductor

In [38]:
DB_DW="dw_multas"
DB_INFO="info_conductores"
DB_MULTAS="multas_transito"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

try:
    dw = connect(DB_DW)
    cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)

    # Conteo antes
    cur.execute("SELECT COUNT(*) FROM DimConductor;")
    before = cur.fetchone()[0]
    print("📊 DimConductor (ANTES):", before)


    sql = f"""
    INSERT INTO DimConductor
      (licencia_id, nombre, genero, estado_civil, anio_nacimiento,
       ingreso_mensual, casa_propia, actividad_principal, scd_activo)
    SELECT
       ci.licencia_id,
       ci.nombre,
       mc.genero,
       ci.estado_civil,
       mc.anio_nacimiento,
       ig.ingreso_mensual,
       ig.casa_propia,
       ap.actividad_principal,
       1
    FROM {DB_INFO}.conductorinfo ci
    LEFT JOIN {DB_INFO}.ingresosconductor ig
      ON ig.licencia_id = ci.licencia_id
    LEFT JOIN (
       SELECT ac.licencia_id, MIN(ac.tipo) AS actividad_principal
       FROM {DB_INFO}.actividadesconductor ac
       GROUP BY ac.licencia_id
    ) ap
      ON ap.licencia_id = ci.licencia_id
    LEFT JOIN {DB_MULTAS}.conductor mc
      ON mc.licencia_id = ci.licencia_id
    LEFT JOIN DimConductor d
      ON d.licencia_id = ci.licencia_id
    WHERE d.conductor_key IS NULL;
    """

    cur.execute(sql)
    dw.commit()


    cur.execute("SELECT COUNT(*) FROM DimConductor;")
    after = cur.fetchone()[0]
    print("📊 DimConductor (DESPUÉS):", after)
    print("🟣 Filas insertadas en esta corrida:", after - before)


    cur.execute("""
        SELECT licencia_id, nombre, genero, estado_civil, anio_nacimiento,
               ingreso_mensual, casa_propia, actividad_principal
        FROM DimConductor
        ORDER BY licencia_id
        LIMIT 10;
    """)
    print("\n🔎 Preview TOP 10 (DimConductor):")
    for r in cur.fetchall():
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
        print("\n🔒 Conexión cerrada.")
    except:
        pass

✅ Conectado a dw_multas; server: 9.4.0
📊 DimConductor (ANTES): 236
📊 DimConductor (DESPUÉS): 236
🟣 Filas insertadas en esta corrida: 0

🔎 Preview TOP 10 (DimConductor):
('LIC00286241', 'Miguel Martinez', 'Masculino', 'Casado', 1999, Decimal('46375.53'), 0, 'empleo')
('LIC00446377', 'Gabriela Romero', 'Femenino', 'Divorciado', 1973, Decimal('50600.73'), 0, 'empleo')
('LIC00446425', 'Mateo Perez', 'Masculino', 'Viudo', 1997, Decimal('29962.43'), 1, 'empleo')
('LIC00985574', 'Ava Diaz', 'Femenino', 'Divorciado', 2005, Decimal('25732.83'), 1, 'empleo')
('LIC02275811', 'Andrea Romero', 'Femenino', 'Casado', 1987, Decimal('54886.04'), 1, 'empleo')
('LIC02783338', 'Juan Chavez', 'Masculino', 'Soltero', 1987, Decimal('31762.03'), 0, 'empleo')
('LIC03423552', 'David Rivera', 'Masculino', 'Soltero', 1973, Decimal('48380.37'), 0, 'empleo')
('LIC04464900', 'Victoria Mendoza', 'Femenino', 'Soltero', 1959, Decimal('59206.76'), 1, 'empleo')
('LIC05466589', 'Ava Vargas', 'Femenino', 'Viudo', 1983, Dec

## Poblar DimTiempo (calendario desde fechas de multas)

In [39]:
import mysql.connector as mc
from mysql.connector import Error
from datetime import date, timedelta


DB_DW="dw_multas"; DB_MULTAS="multas_transito"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

def daterange(d1, d2):
    """Genera fechas desde d1 hasta d2 (inclusive)."""
    cur = d1
    delta = timedelta(days=1)
    while cur <= d2:
        yield cur
        cur += delta


MESES_ES = ["", "enero","febrero","marzo","abril","mayo","junio",
            "julio","agosto","septiembre","octubre","noviembre","diciembre"]

try:
    dw = connect(DB_DW)
    cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)


    cur.execute(f"""
        SELECT
          LEAST(
            (SELECT MIN(fecha) FROM {DB_MULTAS}.multaconduccion),
            (SELECT MIN(fecha) FROM {DB_MULTAS}.multaestacionamiento)
          ) AS fmin,
          GREATEST(
            (SELECT MAX(fecha) FROM {DB_MULTAS}.multaconduccion),
            (SELECT MAX(fecha) FROM {DB_MULTAS}.multaestacionamiento)
          ) AS fmax;
    """)
    fmin, fmax = cur.fetchone()
    print("📅 Rango detectado:", fmin, "→", fmax)
    if fmin is None or fmax is None:
        raise ValueError("No se detectaron fechas en las tablas de multas.")


    sql = """
    INSERT INTO DimTiempo
      (time_key, fecha, dia, mes, nombre_mes, trimestre, anio, dia_semana, es_fin_semana)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ON DUPLICATE KEY UPDATE time_key = VALUES(time_key);
    """


    batch = []
    for d in daterange(fmin, fmax):
        time_key = int(d.strftime("%Y%m%d"))
        dia = d.day
        mes = d.month
        nombre_mes = MESES_ES[mes]
        trimestre = (mes - 1)//3 + 1
        anio = d.year
        dia_semana = d.weekday() + 1       
        es_fin_semana = 1 if d.weekday() >= 5 else 0
        batch.append((time_key, d, dia, mes, nombre_mes, trimestre, anio, dia_semana, es_fin_semana))


        if len(batch) >= 1000:
            cur.executemany(sql, batch)
            dw.commit()
            batch.clear()

    if batch:
        cur.executemany(sql, batch)
        dw.commit()

    # 4) Conteo + preview
    cur.execute("SELECT COUNT(*) FROM DimTiempo;")
    total = cur.fetchone()[0]
    print("📊 DimTiempo total:", total)

    cur.execute("""
        SELECT time_key, fecha, dia, mes, nombre_mes, trimestre, anio, dia_semana, es_fin_semana
        FROM DimTiempo
        ORDER BY fecha
        LIMIT 7;
    """)
    print("\n🔎 Preview (primeros 7 días):")
    for r in cur.fetchall():
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
        print("\n🔒 Conexión cerrada.")
    except:
        pass


✅ Conectado a dw_multas; server: 9.4.0
📅 Rango detectado: 2021-01-01 → 2025-12-29
📊 DimTiempo total: 1824

🔎 Preview (primeros 7 días):
(20210101, datetime.date(2021, 1, 1), 1, 1, 'enero', 1, 2021, 5, 0)
(20210102, datetime.date(2021, 1, 2), 2, 1, 'enero', 1, 2021, 6, 1)
(20210103, datetime.date(2021, 1, 3), 3, 1, 'enero', 1, 2021, 7, 1)
(20210104, datetime.date(2021, 1, 4), 4, 1, 'enero', 1, 2021, 1, 0)
(20210105, datetime.date(2021, 1, 5), 5, 1, 'enero', 1, 2021, 2, 0)
(20210106, datetime.date(2021, 1, 6), 6, 1, 'enero', 1, 2021, 3, 0)
(20210107, datetime.date(2021, 1, 7), 7, 1, 'enero', 1, 2021, 4, 0)

🔒 Conexión cerrada.


In [40]:
import mysql.connector as mc
from mysql.connector import Error

DB_DW="dw_multas"; DB_SRC="multas_transito"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

def show(title, rows):
    print(f"\n=== {title} ===")
    for r in rows:
        print(r)

try:
    dw = connect(DB_DW); cur = dw.cursor()


    cur.execute("SELECT COUNT(*) FROM DimOficial"); c_of = cur.fetchone()[0]
    cur.execute("SELECT COUNT(*) FROM DimVehiculo"); c_ve = cur.fetchone()[0]
    cur.execute("SELECT COUNT(*) FROM DimConductor"); c_co = cur.fetchone()[0]
    cur.execute("SELECT COUNT(*) FROM DimTipoMulta WHERE categoria='conduccion'"); c_tm = cur.fetchone()[0]
    cur.execute("SELECT COUNT(*) FROM DimTiempo"); c_ti = cur.fetchone()[0]
    print(f"DimOficial={c_of}, DimVehiculo={c_ve}, DimConductor={c_co}, DimTipoMulta(conduccion)={c_tm}, DimTiempo={c_ti}")


    cur.execute(f"""
        SELECT DISTINCT mc.placas
        FROM {DB_SRC}.multaconduccion mc
        LEFT JOIN DimVehiculo dv ON dv.placas = UPPER(TRIM(mc.placas))
        WHERE dv.vehiculo_key IS NULL
        LIMIT 20;
    """)
    missing_placas = cur.fetchall()
    show("PLACAS SIN MATCH EN DimVehiculo (muestra)", missing_placas)


    cur.execute(f"""
        SELECT DISTINCT mc.id_tipo_multa_conduccion
        FROM {DB_SRC}.multaconduccion mc
        LEFT JOIN DimTipoMulta dtm
          ON dtm.categoria='conduccion'
         AND dtm.id_tipo_natural = CAST(mc.id_tipo_multa_conduccion AS CHAR)
        WHERE dtm.tipo_multa_key IS NULL
        LIMIT 20;
    """)
    missing_tipos = cur.fetchall()
    show("TIPOS DE MULTA CONDUCCION SIN MATCH EN DimTipoMulta (muestra)", missing_tipos)


    cur.execute(f"""
        SELECT DISTINCT mc.id_oficial
        FROM {DB_SRC}.multaconduccion mc
        LEFT JOIN DimOficial dof ON dof.id_oficial = mc.id_oficial
        WHERE dof.oficial_key IS NULL
        LIMIT 20;
    """)
    missing_of = cur.fetchall()
    show("OFICIALES SIN MATCH EN DimOficial (muestra)", missing_of)


    cur.execute(f"""
        SELECT DISTINCT mc.fecha
        FROM {DB_SRC}.multaconduccion mc
        LEFT JOIN DimTiempo dt
          ON dt.time_key = CAST(DATE_FORMAT(mc.fecha,'%Y%m%d') AS UNSIGNED)
        WHERE dt.time_key IS NULL
        ORDER BY mc.fecha
        LIMIT 10;
    """)
    missing_dates = cur.fetchall()
    show("FECHAS SIN MATCH EN DimTiempo (muestra)", missing_dates)


    cur.execute(f"""
        SELECT COUNT(*)
        FROM {DB_SRC}.multaconduccion mc
        JOIN DimOficial dof
          ON dof.id_oficial = mc.id_oficial
        LEFT JOIN DimConductor dco
          ON dco.licencia_id = mc.licencia_id
        JOIN DimVehiculo dve
          ON dve.placas = UPPER(TRIM(mc.placas))
        JOIN DimTipoMulta dtm
          ON dtm.categoria='conduccion'
         AND dtm.id_tipo_natural = CAST(mc.id_tipo_multa_conduccion AS CHAR)
        JOIN DimTiempo dti
          ON dti.time_key = CAST(DATE_FORMAT(mc.fecha,'%Y%m%d') AS UNSIGNED);
    """)
    ok_rows = cur.fetchone()[0]
    print(f"\n✅ Filas de multaconduccion que sí matchean TODAS las dimensiones: {ok_rows}")

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
    except:
        pass


DimOficial=120, DimVehiculo=732, DimConductor=236, DimTipoMulta(conduccion)=50, DimTiempo=1824

=== PLACAS SIN MATCH EN DimVehiculo (muestra) ===
('AG46963',)
('AM82909',)
('AO08163',)
('BA14178',)
('BI29999',)
('CA2641',)
('CA3513',)
('CA3970',)
('CA5358',)
('CA6317',)
('CA7032',)
('CA8372',)
('CA8785',)
('CDMX1653',)
('CDMX4165',)
('CDMX4444',)
('CDMX5211',)
('CDMX5948',)
('CDMX8227',)
('CH89392',)

=== TIPOS DE MULTA CONDUCCION SIN MATCH EN DimTipoMulta (muestra) ===

=== OFICIALES SIN MATCH EN DimOficial (muestra) ===

=== FECHAS SIN MATCH EN DimTiempo (muestra) ===

✅ Filas de multaconduccion que sí matchean TODAS las dimensiones: 742


## FactMulta (conducción)

In [41]:
import mysql.connector as mc
from mysql.connector import Error

DB_DW="dw_multas"; DB_SRC="multas_transito"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

try:
    dw = connect(DB_DW); cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)

    cur.execute("SELECT COUNT(*) FROM FactMulta;")
    before = cur.fetchone()[0]
    print("📊 FactMulta (ANTES):", before)

    sql = f"""
    INSERT INTO FactMulta
      (id_multa, time_key, oficial_key, conductor_key, vehiculo_key, tipo_multa_key, importe, multa_count)
    SELECT
      mc.id_multa,
      CAST(DATE_FORMAT(mc.fecha,'%Y%m%d') AS UNSIGNED) AS time_key,
      dof.oficial_key,
      dco.conductor_key,
      dve.vehiculo_key,
      dtm.tipo_multa_key,
      dtm.tarifa,
      1
    FROM {DB_SRC}.multaconduccion mc
    JOIN DimOficial dof
      ON dof.id_oficial = mc.id_oficial
    LEFT JOIN DimConductor dco
      ON dco.licencia_id = mc.licencia_id
    JOIN DimVehiculo dve
      ON dve.placas_norm = REGEXP_REPLACE(UPPER(mc.placas), '[^A-Z0-9]', '')
    JOIN DimTipoMulta dtm
      ON dtm.categoria='conduccion'
     AND dtm.id_tipo_natural = CAST(mc.id_tipo_multa_conduccion AS CHAR)
    JOIN DimTiempo dti
      ON dti.time_key = CAST(DATE_FORMAT(mc.fecha,'%Y%m%d') AS UNSIGNED)
    LEFT JOIN FactMulta fm
      ON fm.id_multa = mc.id_multa
    WHERE fm.fact_multa_id IS NULL;
    """
    cur.execute(sql); dw.commit()

    cur.execute("SELECT COUNT(*) FROM FactMulta;")
    after = cur.fetchone()[0]
    print("📊 FactMulta (DESPUÉS):", after, " | Insertadas (conducción):", after - before)

    cur.execute("""
        SELECT id_multa, time_key, oficial_key, conductor_key, vehiculo_key, tipo_multa_key, importe
        FROM FactMulta
        ORDER BY fact_multa_id DESC
        LIMIT 5;
    """)
    for r in cur.fetchall():
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
    except:
        pass


✅ Conectado a dw_multas; server: 9.4.0
📊 FactMulta (ANTES): 815
📊 FactMulta (DESPUÉS): 815  | Insertadas (conducción): 0
('900', 20220527, 52, None, 388, 67, Decimal('380.00'))
('896', 20210704, 77, None, 730, 101, Decimal('980.00'))
('894', 20240908, 19, None, 636, 74, Decimal('1385.00'))
('868', 20210306, 66, None, 313, 98, Decimal('1890.00'))
('847', 20221128, 23, None, 79, 103, Decimal('2070.00'))


## FactMulta (estacionamiento)

In [42]:
DB_DW="dw_multas"; DB_SRC="multas_transito"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

try:
    dw = connect(DB_DW)
    cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)

    # Conteo antes
    cur.execute("SELECT COUNT(*) FROM FactMulta;")
    before = cur.fetchone()[0]
    print("📊 FactMulta (ANTES):", before)

    # Inserta Estacionamiento (conductor_key = NULL)
    sql = f"""
    INSERT INTO FactMulta
      (id_multa, time_key, oficial_key, conductor_key, vehiculo_key, tipo_multa_key, importe, multa_count)
    SELECT
      me.id_multa,
      CAST(DATE_FORMAT(me.fecha,'%Y%m%d') AS UNSIGNED) AS time_key,
      dof.oficial_key,
      NULL AS conductor_key,
      dve.vehiculo_key,
      dtm.tipo_multa_key,
      dtm.tarifa,
      1
    FROM {DB_SRC}.multaestacionamiento me
    JOIN DimOficial dof
      ON dof.id_oficial = me.id_oficial
    JOIN DimVehiculo dve
      ON dve.placas = UPPER(TRIM(me.placas))
    JOIN DimTipoMulta dtm
      ON dtm.categoria='estacionamiento'
     AND dtm.id_tipo_natural = CAST(me.id_tipo_multa_estacionamiento AS CHAR)
    JOIN DimTiempo dti
      ON dti.time_key = CAST(DATE_FORMAT(me.fecha,'%Y%m%d') AS UNSIGNED)
    LEFT JOIN FactMulta fm
      ON fm.id_multa = me.id_multa
    WHERE fm.fact_multa_id IS NULL;
    """
    cur.execute(sql)
    dw.commit()

    # Conteo después + muestra
    cur.execute("SELECT COUNT(*) FROM FactMulta;")
    after = cur.fetchone()[0]
    print("📊 FactMulta (DESPUÉS):", after)
    print("🟣 Filas insertadas (estacionamiento):", after - before)

    cur.execute("""
        SELECT id_multa, time_key, oficial_key, conductor_key, vehiculo_key, tipo_multa_key, importe
        FROM FactMulta
        ORDER BY fact_multa_id DESC
        LIMIT 5;
    """)
    print("\n🔎 Preview últimas 5 (estacionamiento):")
    for r in cur.fetchall():
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
        print("\n🔒 Conexión cerrada.")
    except:
        pass

✅ Conectado a dw_multas; server: 9.4.0
📊 FactMulta (ANTES): 815
📊 FactMulta (DESPUÉS): 815
🟣 Filas insertadas (estacionamiento): 0

🔎 Preview últimas 5 (estacionamiento):
('900', 20220527, 52, None, 388, 67, Decimal('380.00'))
('896', 20210704, 77, None, 730, 101, Decimal('980.00'))
('894', 20240908, 19, None, 636, 74, Decimal('1385.00'))
('868', 20210306, 66, None, 313, 98, Decimal('1890.00'))
('847', 20221128, 23, None, 79, 103, Decimal('2070.00'))

🔒 Conexión cerrada.


## FactIngresosDiarios (agregado día × oficial)

In [43]:
DB_DW="dw_multas"

def connect(db=None):
    return mc.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=db)

try:
    dw = connect(DB_DW)
    cur = dw.cursor()
    print(f"✅ Conectado a {DB_DW}; server:", dw.server_info)


    sql = """
    INSERT INTO FactIngresosDiarios (time_key, oficial_key, total_multas, total_ingresos)
    SELECT
      fm.time_key,
      fm.oficial_key,
      COUNT(*) AS total_multas,
      SUM(fm.importe) AS total_ingresos
    FROM FactMulta fm
    GROUP BY fm.time_key, fm.oficial_key
    ON DUPLICATE KEY UPDATE
      total_multas = VALUES(total_multas),
      total_ingresos = VALUES(total_ingresos);
    """
    cur.execute(sql)
    dw.commit()

    # Preview
    cur.execute("""
        SELECT time_key, oficial_key, total_multas, total_ingresos
        FROM FactIngresosDiarios
        ORDER BY time_key, oficial_key
        LIMIT 10;
    """)
    print("\n🔎 Preview TOP 10 (FactIngresosDiarios):")
    for r in cur.fetchall():
        print(r)

except Error as e:
    print("❌ Error:", e)
finally:
    try:
        cur.close(); dw.close()
        print("\n🔒 Conexión cerrada.")
    except:
        pass

✅ Conectado a dw_multas; server: 9.4.0

🔎 Preview TOP 10 (FactIngresosDiarios):
(20210106, 6, 1, Decimal('1320.00'))
(20210107, 53, 1, Decimal('690.00'))
(20210108, 107, 1, Decimal('950.00'))
(20210109, 74, 1, Decimal('1320.00'))
(20210114, 32, 1, Decimal('2520.00'))
(20210115, 59, 1, Decimal('2540.00'))
(20210117, 109, 1, Decimal('3515.00'))
(20210118, 69, 1, Decimal('4155.00'))
(20210121, 78, 1, Decimal('610.00'))
(20210124, 40, 1, Decimal('1380.00'))

🔒 Conexión cerrada.
